In [1]:
cd langdata

/projects/metis2/anubha/research/Multilingual-Fig-QA/langdata


In [2]:
#Reading Dataset

In [3]:
ls

en_dev.csv  en_train.csv  hi.csv  id.csv  jv.csv  kn.csv  su.csv  sw.csv


In [4]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

### Experiment 6.2.3

In [5]:
import pandas as pd

In [6]:
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [13]:
#load other data
df = pd.read_csv("hi.csv")
df = df.reindex(df_original.columns, axis=1)
df.head()

,startphrase,ending1,ending2,labels
0,दोस्ती एक फलदार वृक्ष है।,दोस्ती मददगार होती है।,दोस्ती नुकसान दायक होती है।,0
1,दोस्ती एक जहरीला वृक्ष है।,दोस्ती मददगार होती है।,दोस्ती नुकसान दायक होती है।,1
2,कालिदास भारत के शेख्चिली हैं।,कालिदास मुर्ख हैं।,कालिदास बुद्धिमान हैं।,0
3,कालिदास भारत के शेक्सपियर हैं।,कालिदास मुर्ख हैं।,कालिदास बुद्धिमान हैं।,1
4,यह खबर उनके दिल में फूलों की खुशबु लाई।,ख़ुशी की खबर लाई।,दुःख की खबर लाई।,0


In [14]:
#Add id to the loaded data
count =0
c=[]
for i in range(int(df.shape[0]/2)):
    c.append(count)
    c.append(count)
    count+=1
df["id"] = c  

In [15]:
df.head()

,startphrase,ending1,ending2,labels,id
0,दोस्ती एक फलदार वृक्ष है।,दोस्ती मददगार होती है।,दोस्ती नुकसान दायक होती है।,0,0
1,दोस्ती एक जहरीला वृक्ष है।,दोस्ती मददगार होती है।,दोस्ती नुकसान दायक होती है।,1,0
2,कालिदास भारत के शेख्चिली हैं।,कालिदास मुर्ख हैं।,कालिदास बुद्धिमान हैं।,0,1
3,कालिदास भारत के शेक्सपियर हैं।,कालिदास मुर्ख हैं।,कालिदास बुद्धिमान हैं।,1,1
4,यह खबर उनके दिल में फूलों की खुशबु लाई।,ख़ुशी की खबर लाई।,दुःख की खबर लाई।,0,2


In [16]:
#df_original = english data
#df = language data
#k = number of datapoints to be added
#pair = in pair / not

def adding_datapoints(df_original, df, k, pair):
    df_final = pd.DataFrame(columns=df_original.columns)
    if pair:
        df_pair = df.sample(n=k, random_state=42)
        for index, r in df_pair.iterrows():
            r_id = r['id']
            w = df.loc[df['id'] == r_id]
            w.sort_values(by=['labels'], inplace=True)
            w= w.drop(['id'], axis=1)
            df_final = df_final.append(w)
    else:
        df_pair = df.sample(n=k, random_state=42)
        w= df_pair.drop(['id'], axis=1)
        df_final = df_final.append(w)

    df_total = pd.concat([df_original,df_final])
    return df_total

    

In [25]:
# #final dataframe, rows added in the end
# df_total = adding_datapoints(df_original, df, 10, True)
# df_total.tail()

#final dataframe, rows added in the end
df_total = adding_datapoints(df_original, df, 10, False)
df_total.tail()

/tmp/ipykernel_10751/1027847878.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(w)


,startphrase,ending1,ending2,labels
678,नौकर चाय में जैसे नमक डाल लाया था।,नौकर ने बिलकुल बकवास चाय बनाई थी।,नौकर ने बहुत बढ़िया चाय बनाई थी।,0
626,बादलों सा ऊँचा उठो।,खूब तरक्की करो।,थोड़ी तो तरक्की करो।,0
513,युद्ध के मैदान में हमारे सैनिकों ने फ़ूलों पर प...,युद्ध में हमारे सैनिकों ने मुश्किलों से विजय प...,युद्ध में हमारे सैनिकों ने आसानी से विजय प्राप...,1
859,पेंटर ने पूरा घर झोंपड़ी सा चमका दिया।,पेंटर ने बहुत बढ़िया रंग किया।,पेंटर ने बहुत घटिया रंग किया।,1
136,उसके लिए पैसा कमाना ऑक्सीजन था।,पैसे कमाने ने उसे जीवित रखा।,पैसा कमाना उसे मार रहा था।,0


In [26]:
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data_winogrand -> for paired data addition
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data -> for single data addition
winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data_winogrand/'
no_winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data/'
name = "hi_10"
df_total.to_csv(no_winogrand+name+ ".csv", index = False)

### Experiment 6.2.2

In [33]:
import pandas as pd
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [35]:
#load other data
df = pd.read_csv("sw.csv")
df = df.reindex(df_original.columns, axis=1)
df.head()

,startphrase,ending1,ending2,labels
0,Watoto ni malaika,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,0.0
1,Yeye ni mnyama.,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,1.0
2,Maneno yake ni kama kisu,Maneno yako yanaumiza,Maneno yake yanafariji,0.0
3,Maneno yake ni kama kinanda,Maneno yako yanaumiza,Maneno yake yanafariji,1.0
4,Tabasamu lake huangaza chumba kizima,Ana tabasamu zuri,Tabasamu lake linatisha,0.0


In [36]:
 df_total = pd.concat([df_original,df])

In [37]:
df_total.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0.0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1.0
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0.0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1.0
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0.0


In [38]:
print(df_original.shape)
print(df.shape)
print(df_total.shape)

(1458, 4)
(798, 4)
(2256, 4)


In [40]:
file = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/exp6.2.2/'
name = "eng_sw"
df_total.to_csv(file+name+ ".csv", index = False)